## This practical is to collect resources for creating knowledge graph. Our aim is to create a knowledge graph from unstructured data such as text on web. To do that, the first step is to collect resources from web; this is the main objective of this practical.

### As this is going to deal with text data, Natural Language Processing (NLP) techniques will have to be used. It is not necessary to know much about NLP for our task as we aim is not to study NLP. Instead, it is only required to know how to use some NLP tools.

### Under the current Covid-19 pandemic, in this exercise we want to know more in general about the pandemic. For example, what and when are the pandemics happened in the past, i.e., the notable outbreaks? To understand that, we want to crawl the web sources such as news and wikipedia. As an example, in this practical we only look at a wikipedia source: https://en.wikipedia.org/wiki/Pandemic. For crawling more web pages, some tools may have to be used, for example, the NewsAPI (https://newsapi.org/).

### Our goal in this practical is to convert the text in this web site into a Knowledge Graph. The first step is to extract the whole text from this web site. Given this web source, we can use the Requests module to read all the information from this resource. For information how to use Requests, please refer to https://requests.readthedocs.io/en/master/.

In [1]:
import requests
url = 'https://en.wikipedia.org/wiki/Pandemic'
res = requests.get(url)
html_page = res.content


### All the information is obtained, NLP tools now can be used to extract the text.

In [2]:
import bs4
soup = bs4.BeautifulSoup(html_page, 'html.parser')
all_text = soup.get_text() # get all the text.


### We can then make text to sentences and save them to a csv file.

In [3]:
import nltk
nltk.download('punkt')
from nltk import tokenize

sentences = tokenize.sent_tokenize(all_text) # make text to sentences.
#print(sentences)


[nltk_data] Downloading package punkt to /Users/tangzhixi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import csv

with open('output.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['sentence'])
    for row in sentences:
        writer.writerow([str(row)])

### However, you might have noticed that the data need to be cleaned. Some sentences are not informative and thus need to be removed.

### Now try to create a knowledge graph

In [5]:
# import the sentences from .csv file
import pandas as pd
example_sentences = pd.read_csv("output.csv", encoding= 'unicode_escape')
example_sentences.shape
print(example_sentences)

                                               sentence
0     \n\n\n\nPandemic - Wikipedia\n\n\n\n\n\n\n\n\n...
1        For other uses, see Pandemic (disambiguation).
2     Global epidemic of infectious disease\n\n\nChe...
3     Painting by Michel Serre during the Great Plag...
4     [1] Hotels and dormitories were also considere...
...                                                 ...
1167  vteGlobal catastrophic risks\nFuture of the Ea...
1168  Text is available under the Creative Commons A...
1169  By using this site, you agree to the Terms of ...
1170  WikipediaÂ® is a registered trademark of the W...
1171  Privacy policy\nAbout Wikipedia\nDisclaimers\n...

[1172 rows x 1 columns]


### Take a few examples to see the Subject, Predicate and Object

In [6]:
sents = example_sentences['sentence'].sample(2)
print(sents)

493                            27 September 1918. p.Â 3.
150    Previously restricted to the Indian subcontine...
Name: sentence, dtype: object


In [7]:
import spacy
nlp=spacy.load('en_core_web_sm')
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]


ModuleNotFoundError: No module named 'spacy'

In [ ]:
get_entities("This is an example of knowledge graph")

In [ ]:
from tqdm import tqdm
entity_pairs = []
for i in tqdm(example_sentences["sentence"]):    
    if type(i) != float:
        entity_pairs.append(get_entities(i))

In [ ]:
entity_pairs[0:10]

### Extract the predicates

In [ ]:
from spacy.matcher import Matcher 
from spacy.tokens import Span 
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)


In [ ]:
get_relation("This is an example of knowledge graph")

In [ ]:
#relations = [get_relation(i) for i in tqdm(example_sentences['[64]'])]
relations = []
for i in tqdm(example_sentences['sentence']):
    if type(i) != float:
        relations.append(get_relation(i))

In [ ]:
pd.Series(relations).value_counts()[:10]

### Create the triples and store in a dataframe

In [ ]:
subjects = [item[0] for item in entity_pairs]
objects = [item[1] for item in entity_pairs]
df_graph = pd.DataFrame({'subjects':subjects, 'objects':objects, 'edge':relations})
print(df_graph)

### We can use networkx to draw a directed graph for these triples

In [ ]:
import networkx as nx

In [ ]:
G=nx.from_pandas_edgelist(df_graph, "subjects", "objects", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

### We can print the graph

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(df_graph[df_graph['edge']=="clarified"], "subjects", "objects", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
print(G)
plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes
nx.draw(G, with_labels=True, node_color='red', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()